#### Задание на ПР-10:
1. Скачайте датасет с рукописными цифрами. Подойдет MNIST или его часть с не менее чем 10000 изображениями. Проследите чтобы число изображений на 1 цифру было примерно равным.
2. Напишите кодировщик, который будет переводить изображение (того формата, который вы скачали) в числовой вектор.
3. Сформируйте числовой датасет, в последнем столбце которого поставьте метку - цифру, которая изображена.
4. Напишите однослойную нейронную сеть с сигмоидой в качестве функции активации. Число нейронов выберите равным 10, число их входов выберите так, чтобы ваш вектор данных входил в сеть. 
5. Обучите сеть так чтобы возбуждение n-го нейрона соответствовало цифре n (нейроны нумеруем от 0 до 9). 
6. Повторите шаги 4 и 5 для двуслойной нейронной сети с промежуточным слоем из 25 нейронов и разными функциями активации (на ваш выбор).
7. Сформируйте предсказания на 1-слойной и 2-слойной ИНС и посчитайте accuracy в обоих случаях. 
8. Сделайте выводы о работе.

In [1]:
# Импорт библиотек и загрузка данных

# Импортируем библиотеку numpy для работы с массивами и матрицами
import numpy as np

# Импортируем библиотеку pandas для работы с табличными данными
import pandas as pd

# Импортируем модуль random для генерации случайных чисел
import random

# Импортируем модуль math для математических функций
import math

# Функция для загрузки данных из CSV файла
def load_data_from_file(filename):
    # Используем pandas для чтения CSV файла
    # Параметр header=None указывает, что в файле нет строки заголовков
    data_frame = pd.read_csv(filename, header=None)
    # Возвращаем загруженные данные в виде DataFrame
    return data_frame

# Загружаем данные MNIST из файла 'mnist_test.csv'
mnist_dataset = load_data_from_file('mnist_test.csv')

# Выводим сообщение об успешной загрузке данных
print("Данные загружены успешно")

Данные загружены успешно


In [2]:
# Функции для обработки изображений и создания датасета

# Функция для нормализации значений пикселей
def normalize_pixels(pixel_data):
    # Создаем пустой список для хранения нормализованных значений
    normalized_pixels = []
    
    # Проходим по каждому пикселю в массиве pixel_data(784)
    for pixel in pixel_data:
        # Нормализуем значение пикселя: делим на 255
        # Это преобразует значения из диапазона 0-255 в диапазон 0-1
        normalized_pixel = pixel / 255.0
        # Добавляем нормализованное значение в список
        normalized_pixels.append(normalized_pixel)
    
    # Возвращаем список нормализованных пикселей
    return normalized_pixels

# Функция для создания обучающего датасета из загруженных данных
def create_training_dataset(data):
    # Создаем пустой список для векторов признаков (изображений)
    feature_vectors = []
    
    # Создаем пустой список для меток классов (цифр)
    target_labels = []
    
    # Получаем количество строк в данных
    data_length = len(data)
    
    # Проходим по всем строкам данных
    for row_index in range(data_length):
        # Получаем значения текущей строки
        row_values = data.iloc[row_index].values
        
        # Первое значение в строке - метка цифры (0-9)
        # Преобразуем его в целое число
        digit_label = int(row_values[0])
        
        # Остальные значения в строке - пиксели изображения
        # Используем срез [1:] чтобы получить все значения кроме первого
        image_pixels = row_values[1:]
        
        # Нормализуем пиксели изображения
        normalized_vector = normalize_pixels(image_pixels)
        
        # Добавляем нормализованный вектор в список признаков
        feature_vectors.append(normalized_vector)
        
        # Добавляем метку класса в список меток
        target_labels.append(digit_label)
    
    # Преобразуем списки в массивы numpy и возвращаем их
    return np.array(feature_vectors), np.array(target_labels)

# Создаем датасет из загруженных данных MNIST
X_features, y_labels = create_training_dataset(mnist_dataset)

In [3]:
# Анализ данных и разделение на выборки

# Выводим заголовок для анализа распределения цифр
print("Анализ распределения цифр в датасете:")

# Создаем пустой словарь для подсчета количества каждой цифры
digit_distribution = {}

# Проходим по всем меткам в датасете
for digit in y_labels:
    # Проверяем, есть ли уже такая цифра в словаре
    if digit in digit_distribution:
        # Если есть, увеличиваем счетчик на 1
        digit_distribution[digit] = digit_distribution[digit] + 1
    else:
        # Если нет, добавляем новую запись со значением 1
        digit_distribution[digit] = 1

# Получаем отсортированный список ключей словаря
sorted_digits = sorted(digit_distribution.keys())

# Проходим по отсортированным цифрам
for digit in sorted_digits:
    # Получаем количество изображений для текущей цифры
    count = digit_distribution[digit]
    # Выводим информацию о количестве изображений для каждой цифры
    print(f"Цифра {digit}: {count} изображений")

# Функция для разделения данных на обучающую и тестовую выборки
def split_dataset(features, labels, test_fraction=0.2):
    # Получаем общее количество образцов
    total_samples = len(features)
    
    # Вычисляем количество тестовых образцов
    # test_fraction=0.2 означает 20% тестовых данных
    test_samples_count = int(total_samples * test_fraction)
    
    # Создаем список индексов от 0 до total_samples-1
    indices_list = []
    for i in range(total_samples):
        indices_list.append(i)
    
    # Перемешиваем список индексов случайным образом
    # Это важно для случайного распределения данных
    random.shuffle(indices_list)
    
    # Выбираем индексы для тестовой выборки (первые test_samples_count)
    test_indices = indices_list[:test_samples_count]
    
    # Выбираем индексы для обучающей выборки (остальные)
    train_indices = indices_list[test_samples_count:]
    
    # Создаем обучающую выборку признаков по выбранным индексам
    X_train = features[train_indices]
    
    # Создаем тестовую выборку признаков по выбранным индексам
    X_test = features[test_indices]
    
    # Создаем обучающую выборку меток по выбранным индексам
    y_train = labels[train_indices]
    
    # Создаем тестовую выборку меток по выбранным индексам
    y_test = labels[test_indices]
    
    # Возвращаем все четыре выборки
    return X_train, X_test, y_train, y_test

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = split_dataset(X_features, y_labels)

# Выводим размеры обучающей и тестовой выборок
print(f"Обучающая выборка: {len(X_train)} образцов")
print(f"Тестовая выборка: {len(X_test)} образцов")

Анализ распределения цифр в датасете:
Цифра 0: 980 изображений
Цифра 1: 1135 изображений
Цифра 2: 1032 изображений
Цифра 3: 1010 изображений
Цифра 4: 982 изображений
Цифра 5: 892 изображений
Цифра 6: 958 изображений
Цифра 7: 1028 изображений
Цифра 8: 974 изображений
Цифра 9: 1009 изображений
Обучающая выборка: 8000 образцов
Тестовая выборка: 2000 образцов


**Формула обновления весов:** новые_веса = старые_веса - скорость_обучения × градиент

In [4]:
# Функции активации

# Сигмоидная функция активации для скалярных значений
def sigmoid_activation(x):
    # Проверяем, не слишком ли большое значение x
    if x > 100:
        # Если x больше 100, возвращаем 1.0 (сигмоид стремится к 1)
        return 1.0
    # Проверяем, не слишком ли маленькое значение x
    elif x < -100:
        # Если x меньше -100, возвращаем 0.0 (сигмоид стремится к 0)
        return 0.0
    else:
        # Для нормальных значений вычисляем сигмоид по формуле
        return 1.0 / (1.0 + math.exp(-x))

# Производная сигмоидной функции для скалярных значений
def sigmoid_derivative(x):
    # Производная сигмоида вычисляется как f'(x) = f(x) * (1 - f(x))
    return x * (1.0 - x)

# ReLU функция активации для скалярных значений
def relu_activation(x):
    # Проверяем, положительное ли значение x
    if x > 0:
        # Если x положительное, возвращаем x без изменений
        return x
    else:
        # Если x отрицательное или ноль, возвращаем 0.0
        return 0.0

# Производная ReLU функции для скалярных значений
def relu_derivative(x):
    # Проверяем, положительное ли значение x
    if x > 0:
        # Если x положительное, производная равна 1.0
        return 1.0
    else:
        # Если x отрицательное или ноль, производная равна 0.0
        return 0.0

# Векторизованная версия сигмоидной функции для массивов
def vectorized_sigmoid(x_array):
    # Проверяем размерность входного массива
    if x_array.ndim > 1:
        # Если массив двумерный, создаем пустой список для результата
        result = []
        # Получаем количество строк в массиве
        num_rows = x_array.shape[0]
        # Проходим по каждой строке массива
        for row_index in range(num_rows):
            # Получаем текущую строку
            current_row = x_array[row_index]
            # Создаем пустой список для результатов текущей строки
            row_result = []
            # Получаем количество элементов в строке
            row_length = len(current_row)
            # Проходим по каждому элементу строки
            for element_index in range(row_length):
                # Применяем сигмоид к текущему элементу
                element_result = sigmoid_activation(current_row[element_index])
                # Добавляем результат в список строки
                row_result.append(element_result)
            # Добавляем список строки в общий результат
            result.append(row_result)
        # Преобразуем список в массив numpy
        return np.array(result)
    else:
        # Если массив одномерный, создаем пустой список для результата
        result = []
        # Получаем длину массива
        array_length = len(x_array)
        # Проходим по каждому элементу массива
        for element_index in range(array_length):
            # Применяем сигмоид к текущему элементу
            element_result = sigmoid_activation(x_array[element_index])
            # Добавляем результат в список
            result.append(element_result)
        # Преобразуем список в массив numpy
        return np.array(result)

# Векторизованная версия производной сигмоидной функции
def vectorized_sigmoid_derivative(x_array):
    # Проверяем размерность входного массива
    if x_array.ndim > 1:
        # Если массив двумерный, создаем пустой список для результата
        result = []
        # Получаем количество строк в массиве
        num_rows = x_array.shape[0]
        # Проходим по каждой строке массива
        for row_index in range(num_rows):
            # Получаем текущую строку
            current_row = x_array[row_index]
            # Создаем пустой список для результатов текущей строки
            row_result = []
            # Получаем количество элементов в строке
            row_length = len(current_row)
            # Проходим по каждому элементу строки
            for element_index in range(row_length):
                # Применяем производную сигмоида к текущему элементу
                element_result = sigmoid_derivative(current_row[element_index])
                # Добавляем результат в список строки
                row_result.append(element_result)
            # Добавляем список строки в общий результат
            result.append(row_result)
        # Преобразуем список в массив numpy
        return np.array(result)
    else:
        # Если массив одномерный, создаем пустой список для результата
        result = []
        # Получаем длину массива
        array_length = len(x_array)
        # Проходим по каждому элементу массива
        for element_index in range(array_length):
            # Применяем производную сигмоида к текущему элементу
            element_result = sigmoid_derivative(x_array[element_index])
            # Добавляем результат в список
            result.append(element_result)
        # Преобразуем список в массив numpy
        return np.array(result)

# Векторизованная версия ReLU функции
def vectorized_relu(x_array):
    # Проверяем размерность входного массива
    if x_array.ndim > 1:
        # Если массив двумерный, создаем пустой список для результата
        result = []
        # Получаем количество строк в массиве
        num_rows = x_array.shape[0]
        # Проходим по каждой строке массива
        for row_index in range(num_rows):
            # Получаем текущую строку
            current_row = x_array[row_index]
            # Создаем пустой список для результатов текущей строки
            row_result = []
            # Получаем количество элементов в строке
            row_length = len(current_row)
            # Проходим по каждому элементу строки
            for element_index in range(row_length):
                # Применяем ReLU к текущему элементу
                element_result = relu_activation(current_row[element_index])
                # Добавляем результат в список строки
                row_result.append(element_result)
            # Добавляем список строки в общий результат
            result.append(row_result)
        # Преобразуем список в массив numpy
        return np.array(result)
    else:
        # Если массив одномерный, создаем пустой список для результата
        result = []
        # Получаем длину массива
        array_length = len(x_array)
        # Проходим по каждому элементу массива
        for element_index in range(array_length):
            # Применяем ReLU к текущему элементу
            element_result = relu_activation(x_array[element_index])
            # Добавляем результат в список
            result.append(element_result)
        # Преобразуем список в массив numpy
        return np.array(result)

# Векторизованная версия производной ReLU функции
def vectorized_relu_derivative(x_array):
    # Проверяем размерность входного массива
    if x_array.ndim > 1:
        # Если массив двумерный, создаем пустой список для результата
        result = []
        # Получаем количество строк в массиве
        num_rows = x_array.shape[0]
        # Проходим по каждой строке массива
        for row_index in range(num_rows):
            # Получаем текущую строку
            current_row = x_array[row_index]
            # Создаем пустой список для результатов текущей строки
            row_result = []
            # Получаем количество элементов в строке
            row_length = len(current_row)
            # Проходим по каждому элементу строки
            for element_index in range(row_length):
                # Применяем производную ReLU к текущему элементу
                element_result = relu_derivative(current_row[element_index])
                # Добавляем результат в список строки
                row_result.append(element_result)
            # Добавляем список строки в общий результат
            result.append(row_result)
        # Преобразуем список в массив numpy
        return np.array(result)
    else:
        # Если массив одномерный, создаем пустой список для результата
        result = []
        # Получаем длину массива
        array_length = len(x_array)
        # Проходим по каждому элементу массива
        for element_index in range(array_length):
            # Применяем производную ReLU к текущему элементу
            element_result = relu_derivative(x_array[element_index])
            # Добавляем результат в список
            result.append(element_result)
        # Преобразуем список в массив numpy
        return np.array(result)

# Softmax функция для многоклассовой классификации
def softmax_activation(vector):
    # Находим максимальное значение в векторе для численной стабильности
    max_value = np.max(vector)
    
    # Создаем пустой список для экспоненцированных значений
    exp_values = []
    
    # Получаем длину вектора
    vector_length = len(vector)
    
    # Проходим по каждому элементу вектора
    for element_index in range(vector_length):
        # Вычисляем экспоненту от разности элемента и максимума
        current_value = vector[element_index]
        exp_value = math.exp(current_value - max_value)
        # Добавляем результат в список
        exp_values.append(exp_value)
    
    # Вычисляем сумму всех экспоненцированных значений
    sum_exp = 0.0
    for exp_val in exp_values:
        sum_exp = sum_exp + exp_val
    
    # Создаем пустой список для результатов softmax
    softmax_result = []
    
    # Проходим по всем экспоненцированным значениям
    for exp_val in exp_values:
        # Вычисляем softmax для каждого значения
        softmax_value = exp_val / sum_exp
        # Добавляем результат в список
        softmax_result.append(softmax_value)
    
    # Преобразуем список в массив numpy
    return np.array(softmax_result)

In [5]:
# Функции для однослойной нейронной сети

# Функция для инициализации параметров однослойной сети
def initialize_single_layer_parameters(input_size, output_size):
    # input_size = 784 (пикселей в изображении), output_size = 10 (цифр для распознавания
    
    # Инициализируем веса случайными маленькими значениями
    # np.random.randn создает массив заданного размера со случайными значениями
    # Умножаем на 0.01 чтобы начальные значения были маленькими
    weights = np.random.randn(input_size, output_size) * 0.01
    
    # Инициализируем смещения нулями
    biases = np.zeros(output_size)
    
    # Возвращаем инициализированные веса и смещения
    return weights, biases

# Функция прямого распространения для однослойной сети
def single_layer_forward_propagation(input_data, weights, biases):
    # Вычисляем взвешенную сумму: матричное умножение входов на веса
    weighted_sum = np.dot(input_data, weights)
    
    # Добавляем смещения к взвешенной сумме
    weighted_sum = weighted_sum + biases
    
    # Применяем сигмоидную функцию активации к результату
    output = vectorized_sigmoid(weighted_sum)
    
    # Возвращаем выход сети
    return output

# Функция обратного распространения для однослойной сети
def single_layer_backward_propagation(input_data, true_labels, network_output, weights, biases, learning_rate):
    # Получаем размер батча (количество образцов)
    batch_size = input_data.shape[0]
    
    # Получаем количество входных признаков
    input_features = input_data.shape[1]
    
    # Получаем количество выходных нейронов
    output_neurons = weights.shape[1]
    
    # Вычисляем ошибку между предсказанием сети и истинными метками
    output_error = network_output - true_labels
    
    # Вычисляем градиент выходного слоя с учетом производной функции активации
    # Умножаем ошибку на производную сигмоида от выхода сети
    output_gradient = output_error * vectorized_sigmoid_derivative(network_output)
    
    # Вычисляем градиенты для весов 
    # Создаем массив для градиентов весов такого же размера как weights
    weight_gradients = np.zeros((input_features, output_neurons))
    
    # Проходим по всем образцам в батче
    for sample_index in range(batch_size):
        # Получаем входные данные для текущего образца (784 значения)
        current_input = input_data[sample_index]
        
        # Получаем градиент для текущего образца (10 значений)
        current_gradient = output_gradient[sample_index]
        
        # Проходим по всем входным признакам (784 раза)
        for feature_index in range(input_features):
            # Проходим по всем выходным нейронам (10 раз)
            for neuron_index in range(output_neurons):
                # Градиент для веса = входной признак × градиент нейрона
                # Это реализация формулы dW = X.T × dZ
                weight_gradients[feature_index, neuron_index] += current_input[feature_index] * current_gradient[neuron_index]
    
    # Делим на размер батча для усреднения градиентов по всем образцам
    weight_gradients = weight_gradients / batch_size
    
    # Вычисляем градиенты для смещений
    bias_gradients = np.zeros(output_neurons)
    
    # Проходим по всем образцам в батче
    for sample_index in range(batch_size):
        # Получаем градиент для текущего образца
        current_gradient = output_gradient[sample_index]
        
        # Проходим по всем выходным нейронам
        for neuron_index in range(output_neurons):
            # Градиент для смещения = градиент нейрона
            # Это реализация формулы db = sum(dZ)
            bias_gradients[neuron_index] += current_gradient[neuron_index]
    
    # Делим на размер батча для усреднения
    bias_gradients = bias_gradients / batch_size
    
    # Обновляем веса с учетом скорости обучения
    # Формула градиентного спуска: W = W - α × dW
    weights = weights - (learning_rate * weight_gradients)
    
    # Обновляем смещения с учетом скорости обучения
    biases = biases - (learning_rate * bias_gradients)
    
    # Возвращаем обновленные веса и смещения
    return weights, biases

# Функция для преобразования меток в one-hot encoding
def convert_to_one_hot(labels, num_classes=10):
    # Получаем количество образцов
    num_samples = len(labels)
    
    # Создаем матрицу нулей размером num_samples x num_classes
    one_hot_labels = np.zeros((num_samples, num_classes))
    
    # Проходим по всем образцам
    for i in range(num_samples):
        # Получаем метку текущего образца
        current_label = labels[i]
        # Устанавливаем 1.0 в столбце, соответствующем метке
        one_hot_labels[i, current_label] = 1.0
    
    # Возвращаем one-hot представление меток
    return one_hot_labels

# Функция для обучения однослойной сети
def train_single_layer_network(training_data, training_labels, input_size, output_size, learning_rate=0.1, epochs=500):
    # Инициализируем параметры сети
    weights, biases = initialize_single_layer_parameters(input_size, output_size)
    
    # Преобразуем метки в one-hot encoding
    one_hot_labels = convert_to_one_hot(training_labels)
    
    # Получаем количество образцов для обучения
    num_samples = len(training_data)
    
    # Цикл обучения по эпохам
    for epoch in range(epochs):
        # Прямой проход: получаем предсказания сети
        predictions = single_layer_forward_propagation(training_data, weights, biases)
        
        # Обратное распространение: обновляем параметры сети
        weights, biases = single_layer_backward_propagation(
            training_data, one_hot_labels, predictions, weights, biases, learning_rate
        )
        
        # Вывод прогресса обучения каждые 100 эпох
        if epoch % 100 == 0:
            # Инициализируем переменную для общей ошибки
            total_error = 0.0
            
            # Вычисляем среднеквадратичную ошибку
            for i in range(num_samples):
                for j in range(output_size):
                    # Вычисляем ошибку для каждого класса и образца
                    error = predictions[i, j] - one_hot_labels[i, j]
                    # Добавляем квадрат ошибки к общей ошибке
                    total_error = total_error + (error * error)
            
            # Вычисляем среднюю ошибку
            mean_error = total_error / (num_samples * output_size)
            
            # Выводим информацию о текущей эпохе и ошибке
            print(f"Эпоха {epoch}, Средняя ошибка: {mean_error:.4f}")
    
    # Возвращаем обученные веса и смещения
    return weights, biases

# Функция для предсказания с использованием однослойной сети
def predict_single_layer(input_data, weights, biases):
    # Выполняем прямой проход для получения вероятностей
    output_probabilities = single_layer_forward_propagation(input_data, weights, biases)
    
    # Создаем пустой список для предсказанных классов
    predicted_classes = []
    
    # Получаем количество образцов
    num_samples = output_probabilities.shape[0]
    
    # Проходим по всем образцам
    for sample_index in range(num_samples):
        # Получаем вероятности для текущего образца
        current_probabilities = output_probabilities[sample_index]
        
        # Инициализируем лучший класс и максимальную вероятность
        best_class = 0
        highest_probability = current_probabilities[0]
        
        # Проходим по всем классам, начиная со второго
        for class_index in range(1, len(current_probabilities)):
            # Сравниваем текущую вероятность с максимальной
            if current_probabilities[class_index] > highest_probability:
                # Обновляем максимальную вероятность и лучший класс
                highest_probability = current_probabilities[class_index]
                best_class = class_index
        
        # Добавляем предсказанный класс в список
        predicted_classes.append(best_class)
    
    # Преобразуем список в массив numpy
    return np.array(predicted_classes)

**Алгоритм обучения:**

Для каждой эпохи (500 раз):
  1. Прямой проход:
     - Вход × Веса + Смещение = Z
     - Сигмоида(Z) = выход (10 вероятностей)
   
  2. Вычисление ошибки:
     - Выход - One-hot метка = ошибка
   
  3. Обратное распространение: (найти градиенты)
     - Градиент = ошибка × производная_сигмоиды
     - dW = входᵀ × градиент / batch_size
     - db = сумма(градиент) / batch_size
   
  4. Обновление весов: (градиентный спуск)
     - Веса = Веса - 0.1 × dW
     - Смещения = Смещения - 0.1 × db

**Предсказание:**
1. Прямой проход 
2. Найти нейрон с максимальным значением 
3. Его индекс = предсказанная цифра

In [6]:
# Функции для двухслойной нейронной сети

# Функция для инициализации параметров двухслойной сети
def initialize_two_layer_parameters(input_size, hidden_size, output_size):
    # Инициализируем веса первого слоя случайными маленькими значениями
    weights1 = np.random.randn(input_size, hidden_size) * 0.01
    
    # Инициализируем смещения первого слоя нулями
    biases1 = np.zeros(hidden_size)
    
    # Инициализируем веса второго слоя случайными маленькими значениями
    weights2 = np.random.randn(hidden_size, output_size) * 0.01
    
    # Инициализируем смещения второго слоя нулями
    biases2 = np.zeros(output_size)
    
    # Возвращаем все инициализированные параметры
    return weights1, biases1, weights2, biases2

# Функция прямого распространения для двухслойной сети
def two_layer_forward_propagation(input_data, weights1, biases1, weights2, biases2):
    # Первый слой: линейное преобразование
    z1 = np.dot(input_data, weights1)
    z1 = z1 + biases1
    
    # Применяем ReLU активацию к первому слою
    a1 = vectorized_relu(z1)
    
    # Второй слой: линейное преобразование
    z2 = np.dot(a1, weights2)
    z2 = z2 + biases2
    
    # Создаем пустой список для выходных вероятностей
    a2_list = []
    
    # Получаем количество образцов
    num_samples = z2.shape[0]
    
    # Проходим по всем образцам
    for sample_index in range(num_samples):
        # Получаем вектор для текущего образца
        current_vector = z2[sample_index]
        
        # Применяем softmax к текущему вектору
        softmax_result = softmax_activation(current_vector)
        
        # Добавляем результат в список
        a2_list.append(softmax_result)
    
    # Преобразуем список в массив numpy
    a2 = np.array(a2_list)
    
    # Возвращаем все промежуточные значения
    return z1, a1, z2, a2

# Функция обратного распространения для двухслойной сети
def two_layer_backward_propagation(input_data, true_labels, z1, a1, z2, a2, weights1, biases1, weights2, biases2, learning_rate):
    # Получаем размер батча (сколько изображений обрабатываем за раз)
    batch_size = input_data.shape[0]
    
    # Получаем размерности для удобства
    num_input_features = input_data.shape[1]  # 784 пикселя
    num_hidden_neurons = weights1.shape[1]    # 25 нейронов в скрытом слое
    num_output_neurons = weights2.shape[1]    # 10 нейронов в выходном слое
    
    # Градиенты для выходного слоя (второго слоя)
    # dz2 = ошибка выходного слоя: предсказание минус правильный ответ
    dz2 = a2 - true_labels
    
    # Градиенты весов второго слоя (dw2) 
    dw2 = np.zeros((num_hidden_neurons, num_output_neurons))
    
    # Проходим по всем образцам в батче
    for i in range(batch_size):
        # Получаем активации скрытого слоя для i-го образца (25 чисел)
        current_a1 = a1[i]
        
        # Получаем градиент выходного слоя для i-го образца (10 чисел)
        current_dz2 = dz2[i]
        
        # Для каждой пары (скрытый нейрон, выходной нейрон) вычисляем градиент
        for hidden_idx in range(num_hidden_neurons):
            for output_idx in range(num_output_neurons):
                # Градиент = активация скрытого нейрона × градиент выходного нейрона
                # Это реализация формулы: dW2[j,k] += a1[i,j] * dz2[i,k]
                dw2[hidden_idx, output_idx] += current_a1[hidden_idx] * current_dz2[output_idx]
    
    # Усредняем градиенты по размеру батча
    dw2 = dw2 / batch_size
    
    # Градиенты смещений второго слоя (db2)
    # db2 должен иметь размер (10,) - как biases2
    db2 = np.zeros(num_output_neurons)
    
    # Проходим по всем образцам в батче
    for i in range(batch_size):
        current_dz2 = dz2[i]
        for output_idx in range(num_output_neurons):
            # Градиент смещения = градиент выходного нейрона
            # Это реализация формулы: db2[k] += dz2[i,k]
            db2[output_idx] += current_dz2[output_idx]
    
    # Усредняем
    db2 = db2 / batch_size
    
    # Градиенты для скрытого слоя (первого слоя)
    # dz1 = градиент для активаций скрытого слоя
    dz1 = np.zeros((batch_size, num_hidden_neurons))
    
    # Проходим по всем образцам
    for i in range(batch_size):
        current_dz2 = dz2[i]  # Градиенты выходного слоя для i-го образца
        current_a1 = a1[i]    # Активации скрытого слоя для i-го образца
        
        # Для каждого нейрона скрытого слоя
        for hidden_idx in range(num_hidden_neurons):
            # Вычисляем сумму: dz2 × веса второго слоя
            # Это часть формулы: dz1 = dz2 × W2^T
            sum_for_neuron = 0.0
            for output_idx in range(num_output_neurons):
                # Берём градиент выходного нейрона и умножаем на соответствующий вес
                # weights2[hidden_idx, output_idx] - вес от hidden_idx к output_idx
                sum_for_neuron += current_dz2[output_idx] * weights2[hidden_idx, output_idx]
            
            # Умножаем на производную ReLU от активации скрытого нейрона
            # relu_derivative возвращает 1 если current_a1[hidden_idx] > 0, иначе 0
            dz1[i, hidden_idx] = sum_for_neuron * relu_derivative(current_a1[hidden_idx])
    
    # Градиенты весов первого слоя
    dw1 = np.zeros((num_input_features, num_hidden_neurons))
    
    # Проходим по всем образцам
    for i in range(batch_size):
        current_input = input_data[i]  # Входные данные i-го образца (784 пикселя)
        current_dz1 = dz1[i]           # Градиент скрытого слоя для i-го образца
        
        # Для каждой пары (входной признак, скрытый нейрон)
        for feature_idx in range(num_input_features):
            for hidden_idx in range(num_hidden_neurons):
                # Градиент = входной признак × градиент скрытого нейрона
                # Это реализация формулы: dW1[j,k] += input[i,j] * dz1[i,k]
                dw1[feature_idx, hidden_idx] += current_input[feature_idx] * current_dz1[hidden_idx]
    
    # Усредняем
    dw1 = dw1 / batch_size
    
    # Градиенты смещений первого слоя (db1)
    db1 = np.zeros(num_hidden_neurons)
    
    # Проходим по всем образцам
    for i in range(batch_size):
        current_dz1 = dz1[i]
        for hidden_idx in range(num_hidden_neurons):
            # Градиент смещения = градиент скрытого нейрона
            db1[hidden_idx] += current_dz1[hidden_idx]
    
    # Усредняем
    db1 = db1 / batch_size
    
    # Обновление параметров второго слоя
    # Формула градиентного спуска: параметр = параметр - скорость_обучения × градиент
    for hidden_idx in range(num_hidden_neurons):
        for output_idx in range(num_output_neurons):
            weights2[hidden_idx, output_idx] = weights2[hidden_idx, output_idx] - (learning_rate * dw2[hidden_idx, output_idx])
    
    for output_idx in range(num_output_neurons):
        biases2[output_idx] = biases2[output_idx] - (learning_rate * db2[output_idx])
    
    # Обновление параметров первого слоя
    for feature_idx in range(num_input_features):
        for hidden_idx in range(num_hidden_neurons):
            weights1[feature_idx, hidden_idx] = weights1[feature_idx, hidden_idx] - (learning_rate * dw1[feature_idx, hidden_idx])
    
    for hidden_idx in range(num_hidden_neurons):
        biases1[hidden_idx] = biases1[hidden_idx] - (learning_rate * db1[hidden_idx])
    
    # Возвращаем обновленные параметры
    return weights1, biases1, weights2, biases2

# Функция для обучения двухслойной сети
def train_two_layer_network(training_data, training_labels, input_size, hidden_size, output_size, learning_rate=0.1, epochs=500):
    # Инициализируем параметры сети
    weights1, biases1, weights2, biases2 = initialize_two_layer_parameters(
        input_size, hidden_size, output_size
    )
    
    # Преобразуем метки в one-hot encoding
    one_hot_labels = convert_to_one_hot(training_labels)
    
    # Получаем количество образцов
    num_samples = len(training_data)
    
    # Цикл обучения по эпохам
    for epoch in range(epochs):
        # Прямой проход через сеть
        z1, a1, z2, a2 = two_layer_forward_propagation(
            training_data, weights1, biases1, weights2, biases2
        )
        
        # Обратное распространение и обновление параметров
        weights1, biases1, weights2, biases2 = two_layer_backward_propagation(
            training_data, one_hot_labels, z1, a1, z2, a2,
            weights1, biases1, weights2, biases2, learning_rate
        )
        
        # Вывод прогресса обучения каждые 100 эпох
        if epoch % 100 == 0:
            # Инициализируем переменную для общей потери
            total_loss = 0.0
            
            # Вычисляем кросс-энтропийную потерю
            for i in range(num_samples):
                # Получаем истинный класс текущего образца
                true_class = training_labels[i]
                
                # Получаем вероятность правильного класса
                correct_probability = a2[i, true_class]
                
                # Защита от нулевой вероятности
                if correct_probability < 1e-10:
                    correct_probability = 1e-10
                
                # Вычисляем логарифм вероятности
                log_prob = math.log(correct_probability)
                
                # Добавляем к общей потере
                total_loss = total_loss - log_prob
            
            # Вычисляем среднюю потерю
            average_loss = total_loss / num_samples
            
            # Выводим информацию о текущей эпохе и потере
            print(f"Эпоха {epoch}, Средние потери: {average_loss:.4f}")
    
    # Возвращаем обученные параметры
    return weights1, biases1, weights2, biases2

# Функция для предсказания с использованием двухслойной сети
def predict_two_layer(input_data, weights1, biases1, weights2, biases2):
    # Выполняем прямой проход через сеть
    z1, a1, z2, a2 = two_layer_forward_propagation(
        input_data, weights1, biases1, weights2, biases2
    )
    
    # Создаем пустой список для предсказанных меток
    predicted_labels = []
    
    # Получаем количество образцов
    num_samples = a2.shape[0]
    
    # Проходим по всем образцам
    for sample_index in range(num_samples):
        # Получаем вероятности для текущего образца
        current_probs = a2[sample_index]
        
        # Инициализируем лучшую метку и максимальную вероятность
        best_label = 0
        max_prob = current_probs[0]
        
        # Проходим по всем классам, начиная со второго
        for label_index in range(1, len(current_probs)):
            # Сравниваем текущую вероятность с максимальной
            if current_probs[label_index] > max_prob:
                # Обновляем максимальную вероятность и лучшую метку
                max_prob = current_probs[label_index]
                best_label = label_index
        
        # Добавляем предсказанную метку в список
        predicted_labels.append(best_label)
    
    # Преобразуем список в массив numpy
    return np.array(predicted_labels)

In [ ]:
# Обучение однослойной сети

# Выводим сообщение о начале обучения
print("Начало обучения однослойной нейронной сети:")

# Получаем размерность входных данных
input_size = X_train.shape[1]

# Определяем количество выходных нейронов (10 цифр)
output_size = 10

# Обучаем однослойную сеть
single_layer_weights, single_layer_biases = train_single_layer_network(
    training_data=X_train,
    training_labels=y_train,
    input_size=input_size,
    output_size=output_size,
    learning_rate=0.1,
    epochs=500
)

# Выводим сообщение об окончании обучения
print("Обучение однослойной сети завершено")

Начало обучения однослойной нейронной сети:
Эпоха 0, Средняя ошибка: 0.2518


**Алгоритм обучения:**

Для каждой эпохи (500 раз):
  1. Прямой проход:
     - Z1 = вход × Веса1 + Смещения1
     - A1 = ReLU(Z1)              ← 25 признаков!
     - Z2 = A1 × Веса2 + Смещения2
     - A2 = Softmax(Z2)           ← 10 вероятностей!
   
  2. Вычисление ошибки:
     - Ошибка = A2 - One-hot метка
   
  3. Обратное распространение: (найти градиенты)
     - dZ2 = ошибка
     - dW2 = A1ᵀ × dZ2 / batch_size
     - db2 = сумма(dZ2) / batch_size
     - dZ1 = (dZ2 × Веса2ᵀ) × производная_ReLU(A1)
     - dW1 = входᵀ × dZ1 / batch_size
     - db1 = сумма(dZ1) / batch_size
   
  4. Обновление весов: (градиентный спуск)
     - Веса2 = Веса2 - 0.1 × dW2
     - Смещения2 = Смещения2 - 0.1 × db2
     - Веса1 = Веса1 - 0.1 × dW1
     - Смещения1 = Смещения1 - 0.1 × db1

In [ ]:
# Обучение двухслойной сети

# Выводим сообщение о начале обучения
print("Начало обучения двухслойной нейронной сети:")

# Получаем размерность входных данных
input_size = X_train.shape[1]

# Определяем количество нейронов в скрытом слое (25 как в задании)
hidden_size = 25

# Определяем количество выходных нейронов (10 цифр)
output_size = 10

# Обучаем двухслойную сеть
two_layer_weights1, two_layer_biases1, two_layer_weights2, two_layer_biases2 = train_two_layer_network(
    training_data=X_train,
    training_labels=y_train,
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    learning_rate=0.1,
    epochs=500
)

# Выводим сообщение об окончании обучения
print("Обучение двухслойной сети завершено")

In [ ]:
# Оценка точности и сравнение моделей

# Функция для вычисления точности предсказаний
def calculate_accuracy(true_labels, predicted_labels):
    # Инициализируем счетчик правильных предсказаний
    correct_predictions = 0
    
    # Получаем общее количество предсказаний
    total_predictions = len(true_labels)
    
    # Проходим по всем образцам
    for i in range(total_predictions):
        # Сравниваем истинную метку с предсказанной
        if true_labels[i] == predicted_labels[i]:
            # Если совпадают, увеличиваем счетчик
            correct_predictions = correct_predictions + 1
    
    # Вычисляем точность как отношение правильных к общему количеству
    accuracy = correct_predictions / total_predictions
    
    # Возвращаем точность
    return accuracy

# Выводим заголовок для оценки точности
print("Оценка точности моделей на тестовой выборке:")

# Получаем предсказания однослойной сети на тестовых данных
y_pred_single = predict_single_layer(
    input_data=X_test,
    weights=single_layer_weights,
    biases=single_layer_biases
)

# Вычисляем точность однослойной сети
accuracy_single = calculate_accuracy(y_test, y_pred_single)

# Выводим точность однослойной сети с округлением до 4 знаков после запятой
print(f"Точность однослойной сети: {accuracy_single:.4f}")

# Получаем предсказания двухслойной сети на тестовых данных
y_pred_two = predict_two_layer(
    input_data=X_test,
    weights1=two_layer_weights1,
    biases1=two_layer_biases1,
    weights2=two_layer_weights2,
    biases2=two_layer_biases2
)

# Вычисляем точность двухслойной сети
accuracy_two = calculate_accuracy(y_test, y_pred_two)

# Выводим точность двухслойной сети с округлением до 4 знаков после запятой
print(f"Точность двухслойной сети: {accuracy_two:.4f}")

# Выводим заголовок для сравнения результатов
print("\nСравнение результатов:")

# Сравниваем точности двух моделей
if accuracy_two > accuracy_single:
    # Если двухслойная сеть лучше, вычисляем процент улучшения
    improvement = ((accuracy_two - accuracy_single) / accuracy_single) * 100
    # Выводим результат сравнения
    print(f"Двухслойная сеть лучше на {improvement:.2f}%")
elif accuracy_single > accuracy_two:
    # Если однослойная сеть лучше, вычисляем процент улучшения
    improvement = ((accuracy_single - accuracy_two) / accuracy_two) * 100
    # Выводим результат сравнения
    print(f"Однослойная сеть лучше на {improvement:.2f}%")
else:
    # Если точности равны, выводим соответствующее сообщение
    print("Обе сети показали одинаковую точность")

In [ ]:
# Визуализация примеров и анализ ошибок

# Функция для визуализации примеров предсказаний
def visualize_predictions(features, true_labels, predictions, num_examples=5):
    # Выводим заголовок с количеством примеров
    print(f"\nВизуализация {num_examples} случайных примеров:")
    
    # Проходим по заданному количеству примеров
    for example in range(num_examples):
        # Генерируем случайный индекс из тестовой выборки
        random_index = random.randint(0, len(features) - 1)
        
        # Получаем истинную цифру для выбранного индекса
        true_digit = true_labels[random_index]
        
        # Получаем предсказанную цифру для выбранного индекса
        predicted_digit = predictions[random_index]
        
        # Определяем, правильно ли предсказана цифра
        if true_digit == predicted_digit:
            result = "Правильно"
        else:
            result = "Ошибка"
        
        # Выводим информацию о примере
        print(f"Пример {example + 1}: Истинная цифра = {true_digit}, " +
              f"Предсказанная цифра = {predicted_digit} {result}")

# Визуализируем примеры для однослойной сети
print("Однослойная сеть:")
visualize_predictions(X_test, y_test, y_pred_single)

# Визуализируем примеры для двухслойной сети
print("\nДвухслойная сеть:")
visualize_predictions(X_test, y_test, y_pred_two)

# Функция для анализа ошибок по классам
def analyze_errors(true_labels, predictions):
    # Создаем пустой словарь для анализа ошибок
    error_analysis = {}
    
    # Получаем количество образцов
    num_samples = len(true_labels)
    
    # Проходим по всем образцам
    for i in range(num_samples):
        # Получаем истинную и предсказанную метки
        true = true_labels[i]
        pred = predictions[i]
        
        # Проверяем, является ли предсказание ошибочным
        if true != pred:
            # Создаем ключ из пары (истинная цифра, ошибочное предсказание)
            key = (true, pred)
            
            # Проверяем, есть ли уже такая ошибка в словаре
            if key in error_analysis:
                # Если есть, увеличиваем счетчик
                error_analysis[key] = error_analysis[key] + 1
            else:
                # Если нет, добавляем новую запись
                error_analysis[key] = 1
    
    # Выводим заголовок для анализа ошибок
    print("\nАнализ наиболее частых ошибок:")
    
    # Преобразуем словарь в список кортежей для сортировки
    error_items = []
    for key, value in error_analysis.items():
        error_items.append((key, value))
    
    # Сортируем ошибки по количеству в убывающем порядке
    # Используем ключевую функцию, которая возвращает второй элемент кортежа (количество)
    def get_count(item):
        return item[1]
    
    sorted_errors = sorted(error_items, key=get_count, reverse=True)
    
    # Выводим 5 самых частых ошибок
    for i in range(min(5, len(sorted_errors))):
        error_pair, count = sorted_errors[i]
        true_digit, wrong_digit = error_pair
        print(f"Цифра {true_digit} ошибочно предсказана как {wrong_digit}: {count} раз")

# Анализируем ошибки однослойной сети
print("Ошибки однослойной сети:")
analyze_errors(y_test, y_pred_single)

# Анализируем ошибки двухслойной сети
print("\nОшибки двухслойной сети:")
analyze_errors(y_test, y_pred_two)

In [ ]:
# Выводы

# Выводим разделительную линию
print("=" * 50)

# Выводим заголовок заключения
print("ЗАКЛЮЧЕНИЕ И ВЫВОДЫ")

# Выводим разделительную линию
print("=" * 50)

# Выводим результаты экспериментов
print("Результаты экспериментов с нейронными сетями:")

# Выводим точность однослойной сети в процентах
print(f"1. Однослойная сеть (сигмоида): {accuracy_single*100:.2f}% точности")

# Выводим точность двухслойной сети в процентах
print(f"2. Двухслойная сеть (ReLU + Softmax): {accuracy_two*100:.2f}% точности")

# Анализируем, какая сеть показала лучший результат
if accuracy_two > accuracy_single:
    # Вычисляем абсолютное улучшение точности
    improvement = accuracy_two - accuracy_single
    
    # Выводим информацию о том, что двухслойная сеть лучше
    print(f"\nДвухслойная сеть показала лучший результат.")
    
    # Выводим процентное улучшение точности
    print(f"Улучшение точности: {improvement*100:.2f}%")
    
    # Объясняем причину улучшения
    print("Это объясняется более сложной архитектурой сети,")
    print("которая может обучаться более сложным признакам.")
    
elif accuracy_single > accuracy_two:
    # Вычисляем абсолютное улучшение точности
    improvement = accuracy_single - accuracy_two
    
    # Выводим информацию о том, что однослойная сеть лучше
    print(f"\nОднослойная сеть показала лучший результат.")
    
    # Выводим процентное улучшение точности
    print(f"Улучшение точности: {improvement*100:.2f}%")
    
    # Объясняем возможные причины
    print("Это может быть связано с особенностями датасета")
    print("или параметрами обучения.")
    
else:
    # Выводим сообщение, если точности равны
    print(f"\nОбе сети показали одинаковую точность.")

# Выводим общие выводы
print("\nОбщие выводы:")

# Вывод 1: обе сети способны обучаться
print("- Обе сети способны обучаться распознаванию рукописных цифр")

# Вывод 2: добавление скрытого слоя может улучшить результаты
print("- Добавление скрытого слоя может улучшить качество классификации")

# Вывод 3: выбор архитектуры зависит от сложности задачи
print("- Выбор архитектуры сети зависит от сложности задачи")

# Вывод 4: для сложных задач нужны более глубокие сети
print("- Для более сложных задач обычно требуются более глубокие сети")